In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert import download
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import os
from transformers import BertModel
from zipfile import ZipFile

In [22]:
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [23]:
def get_pytorch_kobert_model(ctx="cpu", cachedir=".cache"):
    
    def get_kobert_model(model_path, vocab_file, ctx="cpu"):
        bertmodel = BertModel.from_pretrained(model_path)
        device = torch.device(ctx)
        bertmodel.to(device)
        bertmodel.eval()
        vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(
            vocab_file, padding_token="[PAD]"
        )
        return bertmodel, vocab_b_obj

    pytorch_kobert = {
        "url": "s3://skt-lsl-nlp-model/KoBERT/models/kobert_v1.zip",
        "chksum": "411b242919",  # 411b2429199bc04558576acdcac6d498
    }
    
    # download model
    model_info = pytorch_kobert
    model_path, is_cached = download(
        model_info["url"], model_info["chksum"], cachedir=cachedir
    )
    cachedir_full = os.path.expanduser(cachedir)
    zipf = ZipFile(os.path.expanduser(model_path))
    zipf.extractall(path=cachedir_full)
    model_path = os.path.join(os.path.expanduser(cachedir), "kobert_from_pretrained")
    # download vocab
    vocab_path = get_tokenizer()
    return get_kobert_model(model_path, vocab_path, ctx)

In [24]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. C:\Users\WIN10\Documents\GitHub\KoBERT\kobert\.cache\kobert_v1.zip
using cached model. C:\Users\WIN10\Documents\GitHub\KoBERT\kobert\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


In [25]:
df = pd.read_csv(r'C:\Users\WIN10\Documents\GitHub\KoBERT\kobert\input_data.csv',index_col=0)
#df

In [26]:
encoder = LabelEncoder()
encoder.fit(df['상승하락'])
df['상승하락'] = encoder.transform(df['상승하락'])
#df

In [27]:
df['제목'] = df['제목'].astype(str)
df['상승하락'] = df['상승하락'].astype(str)

In [28]:
data = df.values.tolist()
#data

In [29]:
train, test = train_test_split(data, test_size = 0.2, shuffle = False, random_state=777)
print(len(train))
print(len(test))

173617
43405


In [30]:
tokenizer = get_tokenizer()

using cached model. C:\Users\WIN10\Documents\GitHub\KoBERT\kobert\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


In [31]:
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [32]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [40]:
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 8 #64
warmup_ratio = 0.1
num_epochs = 5 #5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-2 # 5e-5


In [34]:
data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

In [35]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size) # num_workers=5
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size) #num_workers=5

In [36]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [37]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [38]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [41]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval() # 평가 모드로 변경
    
    temp_df = pd.DataFrame()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        _,preds = torch.max(out.data, 1)
        
        #temp_df.concat(preds)
        test_acc += calc_accuracy(out, label)
        
    #preds.to_csv(r'C:\Users\WIN10\Documents\GitHub\KoBERT\kobert' + '/' + e + '_preds' + '.csv')    
    #label.to_csv(r'C:\Users\WIN10\Documents\GitHub\KoBERT\kobert' + '/' + e + '_labels' + '.csv')
    print("epoch {} test acc {}, temp_df {}".format(e+1, test_acc / (batch_id+1), temp_df))

epoch 1 batch id 1 loss 5.344594478607178 train acc 0.125
epoch 1 batch id 201 loss 7.07663106918335 train acc 0.4141791044776119
epoch 1 batch id 401 loss 4.286240577697754 train acc 0.5090399002493765
epoch 1 batch id 601 loss 7.69052267074585 train acc 0.5324459234608985
epoch 1 batch id 801 loss 10.152761459350586 train acc 0.533083645443196
epoch 1 batch id 1001 loss 0.001449945499189198 train acc 0.5555694305694305
epoch 1 batch id 1201 loss 0.0023461789824068546 train acc 0.5600541215653622
epoch 1 batch id 1401 loss 3.475349187850952 train acc 0.547644539614561
epoch 1 batch id 1601 loss 0.00077782291918993 train acc 0.5455965021861336
epoch 1 batch id 1801 loss 0.08194095641374588 train acc 0.5468489727928928
epoch 1 batch id 2001 loss 11.463945388793945 train acc 0.5503498250874562
epoch 1 batch id 2201 loss 0.08013486117124557 train acc 0.5557133121308496
epoch 1 batch id 2401 loss 0.0 train acc 0.5550812161599333
epoch 1 batch id 2601 loss 0.0 train acc 0.5590157631680123
e

AttributeError: 'Tensor' object has no attribute 'to_csv'